In [1]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np

In [32]:
# Load dataset
x_train = np.load('E:\whistle_identification\TrainAndVal_vanilla\\trainImages_vanilla_48kHz.npy',allow_pickle=True)
y_train = np.load('E:\whistle_identification\TrainAndVal_vanilla\\trainLabels_vanilla_48kHz.npy',allow_pickle=True)
x_val = np.load('E:\whistle_identification\TrainAndVal_vanilla\\valImages_vanilla_48kHz.npy',allow_pickle=True)
y_val = np.load('E:\whistle_identification\TrainAndVal_vanilla\\valLabels_vanilla_48kHz.npy',allow_pickle=True)
x_test = np.load('E:\whistle_identification\DOLPHI\\total_testX.npy',allow_pickle=True)
y_test = np.load('E:\whistle_identification\DOLPHI\\total_testY.npy',allow_pickle=True)


HP_NODES_CONV = hp.HParam('num_nodes_conv', hp.Discrete([16, 32, 64]))
HP_NODES_DENSE = hp.HParam('num_nodes_dense', hp.Discrete([8, 64, 512]))
HP_BATCH = hp.HParam('batch_size', hp.Discrete([8, 16, 32]))
HP_LRATE = hp.HParam('learning_rate', hp.Discrete([1e-5, 1e-4, 1e-3, 1e-2]))
HP_EPOCHS = hp.HParam('epochs', hp.Discrete([1, 5, 10]))

METRIC_ACCURACY = 'accuracy'




In [1]:
%load_ext tensorboard

In [33]:

with tf.summary.create_file_writer('logs4\hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NODES_CONV, HP_NODES_DENSE, HP_BATCH, HP_LRATE, HP_EPOCHS],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')]
  )


In [34]:

def train_test_model(hparams):
  model = Sequential()
  model.add(Conv2D(hparams[HP_NODES_CONV], (7, 7), strides=(2, 2), activation='relu', input_shape=(224,223,3)))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2))

  model.add(Conv2D(hparams[HP_NODES_CONV]*2, (5, 5), strides=(2, 2), activation='relu')) 
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2))
         
  model.add(Flatten())

  model.add(Dense(hparams[HP_NODES_DENSE], activation='relu'))
  model.add(Dense(hparams[HP_NODES_DENSE]/2, activation='relu'))
  
  # Output layer
  model.add(Dense(1, activation='sigmoid'))

  opt = tf.keras.optimizers.Adam(learning_rate=hparams[HP_LRATE])
  model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])
  model.fit(x_train, y_train, epochs=hparams[HP_EPOCHS], batch_size = hparams[HP_BATCH], validation_data=(x_val,y_val))
  _, accuracy = model.evaluate(x_test, y_test)
  return accuracy

In [35]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)


In [ ]:
session_num = 0
for epochs in HP_EPOCHS.domain.values:
   for num_nodes_dense in HP_NODES_DENSE.domain.values:
      for num_nodes_conv in HP_NODES_CONV.domain.values:
        for learning_rate in HP_LRATE.domain.values:
          for batch_size in HP_BATCH.domain.values:
            hparams = {
              HP_NODES_CONV: num_nodes_conv,
              HP_NODES_DENSE: num_nodes_dense,
              HP_BATCH: batch_size,
              HP_LRATE: learning_rate,
              HP_EPOCHS: epochs
              }
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('logs4/hparam_tuning/' + run_name, hparams)
            session_num += 1